In [592]:
import sympy as sy
from sympy import symbols, Function, diff, Matrix, MatMul, \
    integrate, Symbol, sin, cos, pi, simplify
from dewloosh.solid.navier.symtools import inv_sym_3x3

In [593]:
# coordinates in a suitable vector space
x, y, z = symbols('x y z', real=True)

# kinematic variables
w0 = Function('w_0')(x, y)
thx = diff(w0, y)
thy = -diff(w0, x)

Displacement Field

In [594]:
u = z * thy
v = -z * thx
w = w0
disps = [u, v, w]
Matrix(disps)

Matrix([
[-z*Derivative(w_0(x, y), x)],
[-z*Derivative(w_0(x, y), y)],
[                  w_0(x, y)]])

Boundary Conditions and Trial Solution

In [595]:
mn = m, n = symbols('m n', integer=True, positive=True)
Cmn = symbols('C_{mn}', real=True)
shape = Lx, Ly = symbols('L_x, L_y', real=True)
Sm, Sn = sin(m * pi * x / Lx), sin(n * pi * y / Ly)
Cm, Cn = cos(m * pi * x / Lx), cos(n * pi * y / Ly)

In [596]:
w0_trial = Cmn * Sm * Sn
thx_trial = diff(w0_trial, y)
thy_trial = -diff(w0_trial, x)
trial = {w0 : w0_trial}
disps_trial = [thx_trial, thy_trial, w0_trial]
Matrix(disps_trial)

Matrix([
[ pi*C_{mn}*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[-pi*C_{mn}*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x],
[          C_{mn}*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)]])

Strain - Displacement Equations

In [597]:
exx = diff(u, x)
eyy = diff(v, y)
exy = diff(u, y) + diff(v, x)
exz = diff(w, x) + diff(u, z)
eyz = diff(w, y) + diff(v, z)
strains = [exx, eyy, exy, exz, eyz]
Matrix(strains)

Matrix([
[-z*Derivative(w_0(x, y), (x, 2))],
[-z*Derivative(w_0(x, y), (y, 2))],
[-2*z*Derivative(w_0(x, y), x, y)],
[                               0],
[                               0]])

In [598]:
strains_trial = [s.subs(trial).expand().doit() for s in strains]
Matrix(strains_trial)

Matrix([
[     pi**2*C_{mn}*m**2*z*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2],
[     pi**2*C_{mn}*n**2*z*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[-2*pi**2*C_{mn}*m*n*z*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y)],
[                                                              0],
[                                                              0]])

A possible definition of curvatures would be:

In [599]:
kxx = diff(exx, z)
kyy = diff(eyy, z)
kxy = diff(exy, z)
gen_curvatures = [kxx, kyy, kxy, exz, eyz]
Matrix(gen_curvatures)

Matrix([
[-Derivative(w_0(x, y), (x, 2))],
[-Derivative(w_0(x, y), (y, 2))],
[-2*Derivative(w_0(x, y), x, y)],
[                             0],
[                             0]])

In [600]:
gen_curvatures_trial = [k.subs(trial).expand().doit() for k in gen_curvatures]
Matrix(gen_curvatures_trial)

Matrix([
[     pi**2*C_{mn}*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2],
[     pi**2*C_{mn}*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[-2*pi**2*C_{mn}*m*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y)],
[                                                            0],
[                                                            0]])

strains '+' Hooke model -> stresses

In [601]:
C11, C22, C12, C16, C26, C66 = C126ij = \
    symbols('C_11 C_22 C_12 C_16 C_26 C_66', real=True)
C126 = sy.Matrix([[C11, C12, 0], [C12, C22, 0], [0, 0, C66]])
sxx, syy, sxy = MatMul(C126, sy.Matrix([exx, eyy, exy])).doit()
C44, C55 = C45ij = symbols('C_44 C_55', real=True)
C45 = sy.Matrix([[C44, 0], [0, C55]])
syz, sxz = MatMul(C45, sy.Matrix([eyz, exz])).doit()

# integrate through the thickness
h = Symbol('t', real = True)  # thickness
Int = lambda expr : integrate(expr, (z, -h/2, h/2))
D11, D22, D12, D66 = Dij = symbols('D_11 D_22 D_12 D_66', real = True)
S44, S55, = Sij = symbols('S_44 S_55', real = True)
#
mx, my, mxy = M = Matrix([Int(s * z) for s in [sxx, syy, sxy]])
vx, vy = V = Matrix([Int(s) for s in [sxz, syz]])
#
mx = mx.simplify().expand()
cD11 = mx.coeff(kxx)
cD12 = mx.coeff(kyy)
mx = D11 * kxx + D12 * kyy
#
my = my.simplify().expand()
cD22 = my.coeff(kyy)
cD21 = my.coeff(kxx)
my = D22 * kyy + D12 * kxx
#
mxy = mxy.simplify().expand()
cD66 = mxy.coeff(kxy)
mxy = D66 * kxy
#
vx = vx.simplify().expand()
cS44 = vx.coeff(exz)
vx = S44 * exz
#
vy = vy.simplify().expand()
cS55 = vy.coeff(eyz)
vy = S55 * eyz

mx = mx.simplify().expand()
my = my.simplify().expand()
mxy = mxy.simplify().expand()
vx = vx.simplify().expand()
vy = vy.simplify().expand()

In [602]:
gen_forces = [mx, my, mxy, vx, vy]
Matrix(gen_forces)

Matrix([
[-D_11*Derivative(w_0(x, y), (x, 2)) - D_12*Derivative(w_0(x, y), (y, 2))],
[-D_12*Derivative(w_0(x, y), (x, 2)) - D_22*Derivative(w_0(x, y), (y, 2))],
[                                     -2*D_66*Derivative(w_0(x, y), x, y)],
[                                                                       0],
[                                                                       0]])

In [603]:
gen_forces_trial = [k.subs(trial).expand().doit() for k in gen_forces]
Matrix(gen_forces_trial)

Matrix([
[pi**2*C_{mn}*D_11*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2 + pi**2*C_{mn}*D_12*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[pi**2*C_{mn}*D_12*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2 + pi**2*C_{mn}*D_22*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[                                                           -2*pi**2*C_{mn}*D_66*m*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y)],
[                                                                                                                            0],
[                                                                                                                            0]])

In [604]:
D = sy.zeros(5, 5)
D[0, 0] = mx.coeff(kxx)
D[0, 1] = mx.coeff(kyy)
D[0, 2] = mx.coeff(kxy)
D[1, 0] = my.coeff(kxx)
D[1, 1] = my.coeff(kyy)
D[1, 2] = my.coeff(kxy)
D[2, 0] = mxy.coeff(kxx)
D[2, 1] = mxy.coeff(kyy)
D[2, 2] = mxy/kxy
D[3, 3] = vx.coeff(exz)
D[3, 4] = vx.coeff(eyz)
D[4, 3] = vy.coeff(exz)
D[4, 4] = vy.coeff(eyz)
D

Matrix([
[D_11, D_12,    0, 0, 0],
[D_12, D_22,    0, 0, 0],
[   0,    0, D_66, 0, 0],
[   0,    0,    0, 0, 0],
[   0,    0,    0, 0, 0]])

 Equilibrium equations. The signs of the equations
 is selected such, that the coefficients of the load functions on the
 right-hand sides admit positive signs according to global axes.

In [605]:
# vertical equilibrium
lhs = simplify(-diff(diff(mx, x), x) - 2 * diff(diff(mxy, x),y) - diff(diff(my, y), y)).expand()

In [606]:
lhs

D_11*Derivative(w_0(x, y), (x, 4)) + 2*D_12*Derivative(w_0(x, y), (x, 2), (y, 2)) + D_22*Derivative(w_0(x, y), (y, 4)) + 4*D_66*Derivative(w_0(x, y), (x, 2), (y, 2))

In [607]:
eq_trial = lhs.subs(trial).expand().doit()
eq_trial

pi**4*C_{mn}*D_11*m**4*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**4 + 2*pi**4*C_{mn}*D_12*m**2*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/(L_x**2*L_y**2) + pi**4*C_{mn}*D_22*n**4*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**4 + 4*pi**4*C_{mn}*D_66*m**2*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/(L_x**2*L_y**2)

Substitute trial solution

In [608]:
eq_trial = eq_trial / (Sm * Sn)
eq_trial = eq_trial.simplify().expand()
eq_trial = eq_trial.coeff(Cmn)
eq_trial

pi**4*D_11*m**4/L_x**4 + 2*pi**4*D_12*m**2*n**2/(L_x**2*L_y**2) + pi**4*D_22*n**4/L_y**4 + 4*pi**4*D_66*m**2*n**2/(L_x**2*L_y**2)

In [609]:
from sympy import pycode
pycode(eq_trial)

'math.pi**4*D_11*m**4/L_x**4 + 2*math.pi**4*D_12*m**2*n**2/(L_x**2*L_y**2) + math.pi**4*D_22*n**4/L_y**4 + 4*math.pi**4*D_66*m**2*n**2/(L_x**2*L_y**2)'

In [610]:
from sympy import lambdify

lhs = lambdify([m, n, D11, D12, D22, D66, Lx, Ly], eq_trial, 'numpy')

In [611]:
lhs(1, 1, 1, 1, 1, 1, 1, 1)

779.2727282720193

Postproc

In [612]:
mx = mx.subs(trial).expand().doit()
mx = mx.simplify().expand()
my = my.subs(trial).expand().doit()
my = my.simplify().expand()
mxy = mxy.subs(trial).expand().doit()
mxy = mxy.simplify().expand()
vx = vx.subs(trial).expand().doit()
vx = vx.simplify().expand()
vy = vy.subs(trial).expand().doit()
vy = vy.simplify().expand()
kxx = kxx.subs(trial).expand().doit()
kxx = kxx.simplify().expand()
kyy = kyy.subs(trial).expand().doit()
kyy = kyy.simplify().expand()
kxy = kxy.subs(trial).expand().doit()
kxy = kxy.simplify().expand()
exz = exz.subs(trial).expand().doit()
exz = exz.simplify().expand()
eyz = eyz.subs(trial).expand().doit()
eyz = eyz.simplify().expand()

In [613]:
qxx, qyy, qz = symbols('q_{xx} q_{yy} q_{z}', real=True)
pxx = qxx * Cm * Sn
pyy = qyy * Sm * Cn
pz = qz * Sm * Sn
vx = pxx + diff(mx, x) + diff(mxy, y)
exz = vx / S44
vx = vx.simplify().expand()
exz = exz.simplify().expand()
vy = pyy + diff(my, y) + diff(mxy, x)
eyz = vy / S55
vy = vy.simplify().expand()
eyz = eyz.simplify().expand()

In [614]:
pproc = Matrix([w0_trial, thx_trial, thy_trial, kxx, 
                kyy, kxy, exz, eyz, mx, my, mxy, vx, vy]).T
pycode(pproc)
pproc.T

Matrix([
[                                                                                                                                                                                                                                        C_{mn}*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)],
[                                                                                                                                                                                                                               pi*C_{mn}*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[                                                                                                                                                                                                                              -pi*C_{mn}*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x],
[                                                                                                                                                                        

In [615]:
gen_forces = [mx, my, mxy, vx, vy]
Matrix(gen_forces)

Matrix([
[                                                                                                                           pi**2*C_{mn}*D_11*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2 + pi**2*C_{mn}*D_12*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[                                                                                                                           pi**2*C_{mn}*D_12*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2 + pi**2*C_{mn}*D_22*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[                                                                                                                                                                                      -2*pi**2*C_{mn}*D_66*m*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y)],
[pi**3*C_{mn}*D_11*m**3*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x**3 + pi**3*C_{mn}*D_12*m*n**2*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/(L_x*L_y**2) + 2*pi**3*C_{mn}*D_66*m*n**2*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/(L_x*L_y**2) + q_{xx}*sin(pi*n*y/L_y)*c

In [616]:
gen_curvatures = [kxx, kyy, kxy, exz, eyz]
Matrix(gen_curvatures)

Matrix([
[                                                                                                                                                                                                                      pi**2*C_{mn}*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2],
[                                                                                                                                                                                                                      pi**2*C_{mn}*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[                                                                                                                                                                                                                 -2*pi**2*C_{mn}*m*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y)],
[pi**3*C_{mn}*D_11*m**3*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/(L_x**3*S_44) + pi**3*C_{mn}*D_12*m*n**2*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/(L_x*L_y**2*S_44) + 2*pi**3*C_{mn}*D_66*

In [617]:
Matrix(disps_trial)

Matrix([
[ pi*C_{mn}*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[-pi*C_{mn}*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x],
[          C_{mn}*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)]])

In [618]:
w, h = symbols('w h', real = True)
xc, yc = symbols('x_c y_c', real = True)

# constans vertical load over a rectangular area
qmn = (4/(Lx*Ly)) * integrate(pz, (x, xc - w/2, xc + w/2),
                            (y, yc - h/2, yc + h/2))
qmn_fz = qmn.simplify().expand()

# constans moment of intensity mx, around global axis X,
# over a rectangular area with width w, height h and center (xc, yc)
qmn = (4/(Lx*Ly)) * integrate(diff(pxx, x), (x, xc - w/2, xc + w/2),
                            (y, yc - h/2, yc + h/2))
qmn_mx = qmn.simplify().expand()

# constant moment of intensity my, around global axis Y,
# over a rectangular area with width w, height h and center (xc, yc)
qmn = (4/(Lx*Ly)) * integrate(diff(pyy, y), (x, xc - w/2, xc + w/2),
                            (y, yc - h/2, yc + h/2))
qmn_my = qmn.simplify().expand()

In [619]:
f = qmn_mx + qmn_my + qmn_fz
f

16*q_{z}*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)/(pi**2*m*n) - 16*q_{yy}*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)/(pi*L_y*m) - 16*q_{xx}*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)/(pi*L_x*n)

In [620]:
rhs = lambdify([m, n, xc, yc, w, h, qxx, qyy, qz, Lx, Ly], f, 'numpy')
rhs(1, 1, 5, 5, 1, 1, 0, 0, 10, 10, 10)

0.3967209360443609

In [621]:
pycode(f)

'16*q_{z}*math.sin((1/2)*math.pi*m*w/L_x)*math.sin(math.pi*m*x_c/L_x)*math.sin((1/2)*math.pi*h*n/L_y)*math.sin(math.pi*n*y_c/L_y)/(math.pi**2*m*n) - 16*q_{yy}*math.sin((1/2)*math.pi*m*w/L_x)*math.sin(math.pi*m*x_c/L_x)*math.sin((1/2)*math.pi*h*n/L_y)*math.sin(math.pi*n*y_c/L_y)/(math.pi*L_y*m) - 16*q_{xx}*math.sin((1/2)*math.pi*m*w/L_x)*math.sin(math.pi*m*x_c/L_x)*math.sin((1/2)*math.pi*h*n/L_y)*math.sin(math.pi*n*y_c/L_y)/(math.pi*L_x*n)'

In [622]:
dofsol = lambdify([m, n, x, y, Lx, Ly, Cmn], Matrix(disps_trial), 'numpy')
dofsol(1, 1, 5, 5, 10, 10, 1)

array([[ 1.92367069e-17],
       [-1.92367069e-17],
       [ 1.00000000e+00]])

In [623]:
Matrix(gen_forces)

Matrix([
[                                                                                                                           pi**2*C_{mn}*D_11*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2 + pi**2*C_{mn}*D_12*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[                                                                                                                           pi**2*C_{mn}*D_12*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2 + pi**2*C_{mn}*D_22*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[                                                                                                                                                                                      -2*pi**2*C_{mn}*D_66*m*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y)],
[pi**3*C_{mn}*D_11*m**3*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x**3 + pi**3*C_{mn}*D_12*m*n**2*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/(L_x*L_y**2) + 2*pi**3*C_{mn}*D_66*m*n**2*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/(L_x*L_y**2) + q_{xx}*sin(pi*n*y/L_y)*c

In [624]:
intfors = lambdify([m, n, x, y, D11, D12, D22, D66, qxx, qyy, Lx, Ly, Cmn], 
                   Matrix(gen_forces), 'numpy')
intfors(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

array([[ 2.96040715e-31],
       [ 2.96040715e-31],
       [-1.97392088e+01],
       [-1.53111597e-14],
       [-1.53111597e-14]])

In [625]:
import numpy as np

size = Lx, Ly = (6, 8)
E = 2.89e7
nu = 0.2
t = 0.25

G = E/2/(1+nu)
D = np.array([[1, nu, 0], [nu, 1, 0], [0., 0, (1-nu)/2]]) * t**3 * (E / (1-nu**2)) / 12
S = np.array([[G, 0], [0, G]]) * t
D11, D12, D22, D66 = D[0, 0], D[0, 1], D[1, 1], D[2, 2]
S44, S55 = S[0, 0], S[1, 1]
xc, yc = size[0]/2, size[1]/2
w, h = size[0], size[1]
m_x, m_y, p_z = 0, 0, -1000

In [626]:
M, N = 50, 50
res = np.zeros((M*N))
for i in range(M):
    for j in range(N):
        b_mn = rhs(i+1, j+1, xc, yc, w, h, m_x, m_y, p_z, Lx, Ly)
        a_mn = lhs(i+1, j+1, D11, D12, D22, D66, Lx, Ly)
        res[i*N + j] = b_mn / a_mn

In [627]:
xp, yp = Lx/2, Ly/2
dp = np.zeros(3)
for i in range(M):
    for j in range(N):
        Cmn = res[i*N + j]
        dp += dofsol(i+1, j+1, xp, yp, Lx, Ly, Cmn).flatten()
dp

array([-4.39385334e-18,  6.89213381e-18, -2.19168691e-01])

In [628]:
from dewloosh.geom import grid, PolyData
from dewloosh.geom.tri.trimesh import triangulate
from dewloosh.geom.topo.tr import Q4_to_T3

shape = nx, ny = (10, 10)
gridparams = {
    'size' : size,
    'shape' : shape,
    'origo' : (0, 0),
    'start' : 0,
    'eshape' : 'Q4'
    }
coords_, topo = grid(**gridparams)
coords = np.zeros((coords_.shape[0], 3))
coords[:, :2] = coords_[:, :]
del coords_
coords, triangles = Q4_to_T3(coords, topo)

triobj = triangulate(points=coords[:, :2], triangles=triangles)[-1]
Mesh = PolyData(coords=coords, topo=triangles)
centers = Mesh.centers()

In [629]:
nP = len(centers)
data = np.zeros((nP, 3))
for i in range(M):
    for j in range(N):
        Cmn = res[i*N + j]
        for k in range(nP):
            xp, yp = centers[k, :2]
            data[k] += dofsol(i+1, j+1, xp, yp, Lx, Ly, Cmn).flatten()

In [ ]:
%matplotlib qt

In [ ]:
from dewloosh.geom.tri.triplot import triplot
import matplotlib.pyplot as plt
from matplotlib import gridspec

plt.style.use('default')
fig = plt.figure(figsize=(8, 3))  # in inches
fig.patch.set_facecolor('white')
cmap = 'jet'
gs = gridspec.GridSpec(1, 4)

# mesh
ax = fig.add_subplot(gs[0])
triplot(triobj, ax=ax, fig=fig)
# rotX
ax = fig.add_subplot(gs[1])
triplot(triobj, ax=ax, fig=fig, title=r'$\theta_x$',
        data=data[:, 0], cmap=cmap, axis='off')
# rotY
ax = fig.add_subplot(gs[2])
triplot(triobj, ax=ax, fig=fig, title=r'$\theta_y$',
        data=data[:, 1], cmap=cmap, axis='off')
# w
ax = fig.add_subplot(gs[3])
triplot(triobj, ax=ax, fig=fig, title=r'$w$',
        data=data[:, 2], cmap=cmap, axis='off')

fig.tight_layout()

In [ ]:
nP = len(centers)
sdata = np.zeros((nP, 5))
for i in range(M):
    for j in range(N):
        Cmn = res[i*N + j]
        for k in range(nP):
            xp, yp = centers[k, :2]
            sdata[k] += intfors(i+1, j+1, xp, yp, D11, D12, 
                                D22, D66, m_x, m_y, Lx, Ly, Cmn).flatten()

In [ ]:
fig = plt.figure(figsize=(10, 2))  # in inches
fig.patch.set_facecolor('white')
cmap = 'jet'
gs = gridspec.GridSpec(1, 5)

# rotX
ax = fig.add_subplot(gs[0])
triplot(triobj, ax=ax, fig=fig, title=r'$m_x$',
        data=sdata[:, 0], cmap=cmap, axis='off')
# rotY
ax = fig.add_subplot(gs[1])
triplot(triobj, ax=ax, fig=fig, title=r'$m_y$',
        data=sdata[:, 1], cmap=cmap, axis='off')
# w
ax = fig.add_subplot(gs[2])
triplot(triobj, ax=ax, fig=fig, title=r'$m_{xy}$',
        data=sdata[:, 2], cmap=cmap, axis='off')

# rotY
ax = fig.add_subplot(gs[3])
triplot(triobj, ax=ax, fig=fig, title=r'$v_x$',
        data=sdata[:, 3], cmap=cmap, axis='off')
# w
ax = fig.add_subplot(gs[4])
triplot(triobj, ax=ax, fig=fig, title=r'$v_y$',
        data=sdata[:, 4], cmap=cmap, axis='off')

fig.tight_layout()